In [1]:
import sqlite3
import re
import calendar # for isleap
import datetime
import pandas as pd

In [2]:
conn = sqlite3.connect("money.gnucash.sql.gnucash")
c = conn.cursor()
tables = c.execute("select name from sqlite_master where type='table';").fetchall()

print(sorted(zip(*tables))[0])

('gnclock', 'versions', 'accounts', 'books', 'budgets', 'budget_amounts', 'sqlite_sequence', 'commodities', 'lots', 'prices', 'schedxactions', 'transactions', 'splits', 'billterms', 'customers', 'employees', 'entries', 'invoices', 'jobs', 'orders', 'taxtables', 'taxtable_entries', 'vendors', 'recurrences', 'slots')


In [3]:
split = pd.read_sql_query(con=conn,sql="select value_num from splits where splits.tx_guid like 'ffd5%';")

split.sum()

value_num    0
dtype: int64

In [4]:
tx_guids_raw = c.execute("select distinct tx_guid from splits").fetchall()

tx_guids = list(zip(*tx_guids_raw))[0]

len(tx_guids_raw)

6138

In [5]:
for tx in tx_guids:
    splits_in_tx = pd.read_sql_query("select value_num from splits where tx_guid = '{}';".format(tx), conn)
    if splits_in_tx.sum()[0] != 0:
        print("Found unbalanced tx: {}".format(tx))
        break
print("All {} splits are balanced!".format(len(tx_guids)))

All 6138 splits are balanced!


In [6]:
splits_in_tx

,value_num
0,256
1,-256


In [7]:
def get_acc_guid(acc_name, db, strict_match=False):
    'Get accoung guid based on its name'
    c = db.cursor()
    if not strict_match:
        acc_name = acc_name + "%"
    acc_guid = c.execute("select guid from accounts where name like '{}'".format(acc_name)).fetchall()
    if acc_guid == []:
        return "#NOTFOUND"
    elif len(acc_guid) > 1:
        return "#NOTUNIQUE"
    else:
        return acc_guid[0][0]

In [8]:
# Find all transactions in a given period for a given account
acc_guid = c.execute("select guid from accounts where name like 'Epay%';")

In [9]:
acc_guid.fetchall()

[('fc9a21ae46fd2334a1e2974e339d77fd',)]

In [10]:
get_acc_guid("Epay", conn)

'fc9a21ae46fd2334a1e2974e339d77fd'

In [11]:
get_acc_guid("FIB Maestro", conn)

'1467f03d73c837fe0786a8a63c9db821'

In [12]:
# Find all tx balances for a given account in a given period
beg_date = "20170327220000"
end_date = "20170426220000"
acc_name = "Private"
acc_guid = get_acc_guid(acc_name, conn, True)

# Find all tx in the given period
tx_in_period = pd.read_sql_query("SELECT guid FROM transactions WHERE post_date >= {begd} AND post_date <= {endd};".
                                format(begd=beg_date, endd=end_date), conn)

# Find all splits corresponding to the found tx
sp_in_period = pd.read_sql_query("SELECT value_num, transactions.guid, transactions.post_date, transactions.description FROM transactions, splits \
                                 WHERE transactions.guid = splits.tx_guid AND \
                                       transactions.post_date >= {begd} AND transactions.post_date <= {endd} AND \
                                       splits.account_guid = '{accg}' \
                                 ORDER BY post_date;".
                                format(begd=beg_date, endd=end_date, accg=acc_guid), conn)

debit_bal = sp_in_period[sp_in_period.value_num > 0].value_num.sum()
credit_bal = sp_in_period[sp_in_period.value_num < 0].value_num.sum()
period_bal = debit_bal + credit_bal

In [13]:
sp_in_period

,value_num,guid,post_date,description
0,813410,cad986e32ba389f40fc04eab615cd315,20170327220000,salary
1,-80000,e1de733ca4cfe5db19a01de3bdd8b470,20170328220000,saving (to annual bills)
2,-380000,b4a116f63481e22980101628a52eadbf,20170328220000,saving (to monthly bills without horse)
3,-2880,e69f9d66fb67162db733ec630ca53de9,20170328220000,clothes Myprivateboutique.ch
4,-95010,0bae1db7d3173c2b0969bc4525c5b8b0,20170329220000,taxes federal 2017
5,-95000,4c35f5afd1ff19edef3e1c8686b0494d,20170329220000,CFF credit card installment
6,-10870,38d9e7fc13dcbaedc8abb3179c6abb8c,20170329220000,Denner??????
7,19350,7c69afe25fe3691b79402735290cac92,20170330220000,salary Vimcho
8,-500,f105cf343fb0af285f72e6d57d5c00c5,20170330220000,bank fees
9,-7000,7ae3dd2c28a0ed6f6277b78f912c6937,20170330220000,nails


In [14]:
period_bal

-440

In [15]:
def date_to_gncdate_naive(date_ddmmyy):
    regex = re.compile(r'(\d{1,2}).(\d{1,2}).(\d{2,4})')
    days_in_month = [31, -1, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31] # February's len is calculated below
    
    m = regex.match(date_ddmmyy)
    if m:
        print(m.groups())
        year = int(m.groups()[2])
        if year < 99:
            year += 2000
        if calendar.isleap(year):
            days_in_month[1] = 29
        else:
            days_in_month[1] = 28
        month = int(m.groups()[1])
        if month > 12:
            raise ValueError("Month must be in the range [1, 12]")
        day = int(m.groups()[0])
        if day > days_in_month[month-1]:
            raise ValueError("Month must be in the range [1, {}]".format(days_in_month[month-1]))
        
        print("{:02}.{:02}.{:02}".format(day, month, year))
        print("{y:02}.{m:02}.{d:02}".format(d=day, m=month, y=year))

In [45]:
class GMT1(datetime.tzinfo):
    def utcoffset(self, dt):
        return datetime.timedelta(hours=1) + self.dst(dt)
    def dst(self, dt):
        # DST starts last Sunday in March
        d = datetime.datetime(dt.year, 4, 1)   # ends last Sunday in October
        self.dston = d - datetime.timedelta(days=d.weekday() + 1)
        d = datetime.datetime(dt.year, 11, 1)
        self.dstoff = d - datetime.timedelta(days=d.weekday() + 1)
        if self.dston <=  dt.replace(tzinfo=None) < self.dstoff:
            return datetime.timedelta(hours=1)
        else:
            return datetime.timedelta(0)
    def tzname(self,dt):
        return "GMT +1"

def date_to_gncdate(date_ddmmyy):
    gmt1 = GMT1()
    try:
        d = datetime.datetime.strptime(date_ddmmyy, "%d.%m.%Y")
    except ValueError:
        d = datetime.datetime.strptime(date_ddmmyy, "%d.%m.%y")
    d = d.replace(tzinfo=gmt1)
    #print(d.tzname())
    #print(d.utcoffset())
    #print(d-d.utcoffset())
    d = d - d.utcoffset()
    ret = "{:04}{:02}{:02}{:02}{:02}{:02}".format(d.year, d.month, d.day, d.hour, d.minute, d.second)
    #print(ret)
    return ret

def iso2gnc_date(isodate):
    #if month < 1 or month > 12:
    #    raise ValueError("Month out of range [1, 12]")
    #if year < 2010 or year > 2050:
    #    raise ValueError("Year out of range [2010, 2050]")
    gmt1 = GMT1()
    try:
        d = datetime.datetime.strptime(isodate, "%Y-%m-%d")
    except ValueError:
        d = datetime.datetime.strptime(isodate, "%y-%m-%d")
    d = d.replace(tzinfo=gmt1)
    d = d - d.utcoffset()
    ret = "{:04}{:02}{:02}{:02}{:02}{:02}".format(d.year, d.month, d.day, d.hour, d.minute, d.second)
    return ret
    #if month == 1:
    #    month = 12
    #    year -= 1
    #else:
    #    month -= 1
    #per_beg = datetime.date(year, month, 28).strftime("%d.%m.%Y")
    
    #return date_to_gncdate(per_beg), date_to_gncdate(per_end)

In [43]:
date_to_gncdate("30.06.2017")

'20170629220000'

In [31]:
# Find debit and credit movements to/from a given account

# Algo:
# 1. Find all tx that have this account in their splits -> tx_acc = [tx where tx.splits contain acc]
# 2. For all tx in tx_acc:
#    debits.append([tx.split.value where tx.split.acc != acc && tx.split.value > 0)
#    credits.append([tx.split.value where tx.split.acc != acc && tx.split.value < 0)

def get_acc_movements(gnc_per_beg, gnc_per_end, acc_name, db):
    #gnc_per_beg = date_to_gncdate(period_beg)
    #gnc_per_end = date_to_gncdate(period_end)
    acc_guid = get_acc_guid(acc_name=acc_name, db=conn, strict_match=True)
    
    tx_acc_raw = pd.read_sql_query("SELECT sp.tx_guid FROM transactions AS tr, splits AS sp WHERE\
                                    tr.guid = sp.tx_guid AND sp.account_guid = '{}' AND\
                                    tr.post_date >= '{}' AND tr.post_date <= '{}';".
                                   format(acc_guid, gnc_per_beg, gnc_per_end), con=db)
    #print(tx_acc_raw)
    splits_for_acc = pd.DataFrame()
    for tx in tx_acc_raw.tx_guid:
        #print(tx)
        splits_for_tx = pd.read_sql_query("SELECT value_num, memo, name, account_type FROM splits, accounts AS acc WHERE\
                                           tx_guid = '{}' AND account_guid <> '{}' AND acc.guid = account_guid;".
                                         format(tx, acc_guid), con=db)
        splits_for_acc = splits_for_acc.append(splits_for_tx)
        #print(splits_for_tx)

    #print(splits_for_acc)
    #per_debit_bal = splits_for_acc[splits_for_acc.value_num > 0].value_num.sum()
    #per_credit_bal = splits_for_acc[splits_for_acc.value_num < 0].value_num.sum()
    #print(per_debit_bal+per_credit_bal)
    return splits_for_acc

In [32]:
am = get_acc_movements("28.03.17", "27.04.17", "Private", conn)

In [34]:
get_gnc_fiscal_period(2017, 3)

('20170227230000', '20170326220000')

In [25]:

def get_asset_cf(month, year, asset_accs, db):
    per_beg, per_end = get_gnc_fiscal_period(year, month)
    
    asset_accs = ("Private", "Private-Bills", "Cash CHF", "Epay Microaccount", "SGS credit account")
    
    asset_cf = pd.DataFrame()
    for acc in asset_accs:
        print(acc)
        acc_mv = get_acc_movements(per_beg, per_end, acc, db)
        # Multiply by -1 to make inflows positive
        acc_mv.value_num *= -1
        asset_cf = asset_cf.append(acc_mv)
        
    # Filter out cf's to/from requested accounts
    for acc in asset_accs:
        asset_cf = asset_cf[asset_cf.name != acc]
        
    # Remove the TRADING accounts
    asset_cf = asset_cf[asset_cf.account_type != "TRADING"]
    
    print(asset_cf)
    out2all = asset_cf[asset_cf.value_num < 0]
    out2ast = out2all[out2all.account_type == "ASSET"]
    out2exp = out2all[out2all.account_type == "EXPENSE"]
    out2lbl = out2all[out2all.account_type == "CREDIT"]
    out2lbl = out2lbl.append(out2all[out2all.account_type == "LIABILITY"])    
    out2inc = out2all[out2all.account_type == "INCOME"]
    
    in_all = asset_cf[asset_cf.value_num > 0]
    in_ast = in_all[in_all.account_type == "ASSET"]
    in_exp = in_all[in_all.account_type == "EXPENSE"]
    in_lbl = in_all[in_all.account_type == "CREDIT"]
    in_lbl = in_lbl.append(in_all[in_all.account_type == "LIABILITY"])
    in_inc = in_all[in_all.account_type == "INCOME"]
    
    ret = {}
    ret['out2all'] = out2all
    ret['out2ast'] = out2ast
    ret['out2exp'] = out2exp
    ret['out2lbl'] = out2lbl
    ret['out2inc'] = out2inc
    ret['in_all']  = in_all
    ret['in_ast']  = in_ast
    ret['in_exp']  = in_exp
    ret['in_lbl']  = in_lbl
    ret['in_inc']  = in_inc
    
    return ret

In [26]:
acc = "Private"

priv_cf = get_asset_cf(3, 2017, "Private", conn)

print("{} inflows from Expenses: {: >10.2f}".format(acc, priv_cf['in_exp'].value_num.sum() / 100))
print("{} inflows from Liab:     {: >10.2f}".format(acc, priv_cf['in_lbl'].value_num.sum() / 100))
print("{} inflows from Incomes:  {: >10.2f}".format(acc, priv_cf['in_inc'].value_num.sum() / 100))
print("                              ===========")
print("{} inflows from All:      {: >10.2f}\n".format(acc, priv_cf['in_all'].value_num.sum() / 100))

print("{} outflow to Expenses:   {: >10.2f}".format(acc, priv_cf['out2exp'].value_num.sum() / 100))
print("{} outflow to Liab:       {: >10.2f}".format(acc, priv_cf['out2lbl'].value_num.sum() / 100))
print("{} outflow to Assets:     {: >10.2f}".format(acc, priv_cf['out2ast'].value_num.sum() / 100))
print("{} outflow to Incomes:    {: >10.2f}".format(acc, priv_cf['out2inc'].value_num.sum() / 100))
print("                              ===========")
print("{} outflow to All:        {: >10.2f}\n".format(acc, priv_cf['out2all'].value_num.sum() / 100))


print("{} balance:               {: >10.2f}".format(acc, (priv_cf['in_all'].value_num.sum() + 
                                                          priv_cf['out2all'].value_num.sum()) / 100))

Private
Private-Bills
Cash CHF
Epay Microaccount
SGS credit account
    value_num             memo                  name account_type
0       -2530             food             Groceries      EXPENSE
1       -1390             wine               Alcohol      EXPENSE
0        -300                                Parking      EXPENSE
0      813410                                 Salary       INCOME
0       12375                                 Salary       INCOME
0     -173000                                  Taxes      EXPENSE
0       -4660                               Services      EXPENSE
0      -75000                   Credit Card CFF Visa       CREDIT
0      -12100                              Groceries      EXPENSE
0       -2395             food             Groceries      EXPENSE
1        -745             wine               Alcohol      EXPENSE
2       -2395  baby cosmestics             Cosmetics      EXPENSE
0       -1990                              Equipment      EXPENSE
0       

In [27]:
priv_cf['out2exp'].groupby('name').sum()/100

,value_num
name,
Alcohol,-85.65
Babysitting,-505.90
Billag,-112.75
Bills BG,-89.56
Cable,-57.60
Clothes,-9.00
Cosmetics,-51.85
Decoration,-26.25
Equipment,-56.70


In [242]:
def get_tx_for_period(per_beg, per_end, db):
    gnc_per_beg = iso2gnc_date(per_beg)
    gnc_per_end = iso2gnc_date(per_end)
    tx_for_period = pd.read_sql_query("SELECT guid, currency_guid, post_date FROM transactions WHERE\
                                      post_date >= '{}' AND post_date <= '{}';".
                                      format(gnc_per_beg, gnc_per_end), con=db)
    return tx_for_period
    
def get_tx_splits(tx_guid, db):
    splits_for_tx = pd.read_sql_query("SELECT guid, account_guid, value_num, value_denom FROM splits WHERE\
                                      tx_guid = '{}';".
                                      format(tx_guid), con=db)
    if splits_for_tx.empty:
        raise ValueError("Transaction guid is invalid. There are no splits associated.")
    
    return splits_for_tx
    
def get_gnc_account_guid(name, db):
    c = db.cursor()
    res = db.execute("SELECT guid FROM accounts WHERE name ='{}';".format(name)).fetchall()
    if res is None:
        raise ValueError("Non-existant account name: {}".format(name))
    if len(res) > 1:
        raise ValueError("Account name {} found multiple times in the database".format(name))
    return res[0][0]

def get_gnc_account_name(guid, db):
    c = db.cursor()
    res = db.execute("SELECT name FROM accounts WHERE guid ='{}';".format(guid)).fetchall()
    if res is None:
        raise ValueError("Non-existant account guid: {}".format(name))
    return res[0][0]

def get_gnc_commodity_guid(name, db):
    c = db.cursor()
    res = db.execute("SELECT guid FROM commodities WHERE mnemonic ='{}';".format(name)).fetchall()
    if res is None:
        raise ValueError("Non-existant commodity name: {}".format(name))
    if len(res) > 1:
        raise ValueError("Commodity name {} found multiple times in the database".format(name))
    return res[0][0]
    
def get_gnc_commodity_price(commodity, currency, age, db):
    prices = pd.read_sql_query("SELECT value_num, value_denom, date FROM prices WHERE\
                                commodity_guid = '{}' AND currency_guid = '{}' AND date <= '{}'\
                                ORDER BY date;".
                              format(commodity, currency, age), con=db)
    if prices.empty:
        raise ValueError("No exchange rate found")
        
    p = prices.iloc[-1]
    return pd.to_numeric(p['value_num']), pd.to_numeric(p['value_denom'])
    
def get_acc_cf(per_beg, per_end, accounts, currency, db):
    """Get total cashflow for a set of accounts.
    
    Algorithm:
    tx_of_period = get_tx_for_period(fiscal_period)
    splits = []
    for tx in tx_period:
        # retrieve all splits of the given transaction
        tx_splits = get_tx_splits(tx)
        
        # delete all splits pertaining to the accounts in questions
        tx_splits = del_splits_for_acc(accounts) 
        
        if tx_splits is not empty:
            if the given transaction is not in the desired currency:
                get the closest prior exchange rate for the tx currency
                convert all tx values to the desired currency
            splits.append(tx_splits)
    """
    
    if db is None or accounts is None:
        return None
    
    #gnc_per_beg, gnc_per_end = get_gnc_fiscal_period(year, month)
    
    tx_for_period = get_tx_for_period(per_beg, per_end, db)
    if tx_for_period.empty:
        return None
    
    # Empty dataframe that will accumulate the final account-trimmed splits
    splits = pd.DataFrame()
    
    #print(tx_for_period)
    for _, tx in tx_for_period.iterrows():
        #print(tx)
        tx_splits = get_tx_splits(tx.guid, db)
        
        # Delete splits belonging to the accounts in question
        for acc in accounts:
            acc_guid = get_gnc_account_guid(acc, db)
            #print("Account {} -> {}".format(acc, acc_guid))
            tx_splits = tx_splits[tx_splits.account_guid != acc_guid]
            
        if tx_splits.empty:
            continue
            
        # Apply exchange rate if needed
        wanted_currency_guid = get_gnc_commodity_guid(currency, db)
        if tx.currency_guid != wanted_currency_guid:
            #print(tx.post_date)
            xrate_num, xrate_denom = get_gnc_commodity_price(tx.currency_guid, 
                                                             wanted_currency_guid, 
                                                             tx.post_date, db)
            tx_splits.value_num *= xrate_num
            tx_splits.value_denom *= xrate_denom
            tx.currency_guid = wanted_currency_guid
            
        splits = splits.append(tx_splits)
    
    return splits
    

In [243]:
sp = get_acc_cf("2017-02-28", "2017-03-27", ("Private",), "CHF", conn)
sp['final_val'] = sp.value_num / sp.value_denom
grp = sp.groupby('account_guid').sum()
grp['acc'] = grp.apply(lambda x: get_gnc_account_name(x.name, conn), axis=1)
print(grp[['acc', 'final_val']])

                                                    acc    final_val
account_guid                                                        
0500d5f7bf83dd6ced43fb688aa3d311   Credit Card CFF Visa  -344.410000
065cf464ceffdd68bcbb36b983fee364                  Taxes  1730.000000
07082b725bbfbbfa0d7488d6e24ebf9e                  Cable    57.600000
0796605acc13980eedbd525997e6cbe6               Bills BG    48.962452
0ec685b50c3af945211492af12d09b9e                  Phone    40.000000
1283c17dcdec4dc002f9b40a61d91dfe           Gifts family    68.600000
12885edd8630cc4819c5c7e38fa7cff4                 Billag   112.750000
1706edb22c7673f37843951230899f81                Medical   328.200000
200385629596a56bea17dafddc4aa58f              Education    20.350000
227b62ab58d9dd24c4d375cca1dee38b               Cash CHF   -80.000000
254aed1088a65eff6ba1c6ebdb0a27c8               Vacation    26.490000
26b1b33f04fb20acbde54996fbe10ac4               Loan car   210.263067
29e41a74e02728f32f9fd55a0e61f4b0  